In [113]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm


In [114]:
def calculate_loadprofiles(df):
    # set index to timestamp
    df.set_index('timestamp', inplace=True) 
    loadprofiles = pd.DataFrame(columns=["residential_id", "daily", "weekly", "monthly"])

    for i in tqdm(df.residential_id.unique()):
        house = df[df.residential_id == i]
        daily = house.resample('D').sum()

        # calculate loadprofiles
        lp_daily = house["energy"].groupby(house.index.hour).mean().values.flatten().tolist()
        lp_weekly = daily["energy"].groupby(daily.index.dayofweek).mean().values.flatten().tolist()
        lp_monthly = daily["energy"].groupby(daily.index.day).mean().values.flatten().tolist()

        # create new row
        data = {"residential_id": [i], "daily": [lp_daily], "weekly": [lp_weekly], "monthly": [lp_monthly]}
        new_row = pd.DataFrame(data)

        # concat new row to loadprofiles df
        loadprofiles = pd.concat([loadprofiles, new_row], ignore_index=True)
    return loadprofiles

In [115]:
# data on the compute machine in energy-forecast-compressed.zip

meta_data = pd.read_parquet("../data/processed/HUE_metadata.parquet")

residentials = pd.read_parquet("../data/processed/HUE_residentials.parquet")

In [116]:
# wH -> kWh
residentials["energy"] = residentials["energy"] /1000



In [117]:
df = residentials.copy()

loadprofiles = calculate_loadprofiles(df)



  0%|          | 0/27 [00:00<?, ?it/s]

100%|██████████| 27/27 [00:00<00:00, 179.55it/s]


In [118]:

loadprofiles.reset_index(drop=True, inplace=True)
loadprofiles.set_index('residential_id', inplace=True)
loadprofiles.sort_index(inplace=True)
loadprofiles.head()


,daily,weekly,monthly
residential_id,,,
1,"[0.876070549630845, 0.6444916387959866, 0.6267...","[26.961379310344824, 25.69296551724138, 25.912...","[26.874073170731705, 26.441536585365856, 25.18..."
2,"[0.21031897926634768, 0.19093600000000002, 0.1...","[8.442166666666667, 8.135666666666667, 8.12283...","[9.228048780487805, 9.123414634146341, 8.05317..."
3,"[0.5134072022160665, 0.5068474264705882, 0.504...","[21.039745222929934, 21.857898089171975, 21.16...","[23.271666666666665, 21.75, 22.574722222222224..."
4,"[0.8284770642201834, 0.7468675799086758, 0.705...","[29.378152866242036, 29.085128205128203, 29.05...","[29.767500000000002, 29.824444444444445, 28.84..."
5,"[0.4894490358126722, 0.4128310502283105, 0.387...","[18.316624203821657, 17.56051282051282, 18.336...","[18.683611111111112, 17.485555555555557, 18.55..."


In [119]:
loadprofiles.to_parquet("../data/processed/HUE_loadprofiles.parquet")